In [1]:
from gensim.models import KeyedVectors
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

In [2]:
model=KeyedVectors.load_word2vec_format("wiki.pa.vec")

In [3]:
vocab=[]
for word in model.vocab:
    vocab.append(word)

In [4]:
# model.vocab[search]
# model.get_vector(search).shape

In [5]:
len(vocab)

56850

In [6]:
vocab[10:21]

['ਦਾ', '}', 'ਅਤੇ', 'ਹੈ।', 'ਨੂੰ', 'ਹੈ', 'ਇੱਕ', 'the', 'ਤੋਂ', 'ਇਸ', 'ਸ਼੍ਰੇਣੀ']

In [16]:
stopwords = ['ਹੋਣ', 'ਆਪਣੇ', 'ਕਰਕੇ', 'ਲਿਆ', 'ਹੋਇਆ', 'ਤੱਕ', 'ਲੈ', 'ਰਹੇ', 'ਦੱਸਿਆ', 'ਪੀ', 'ਕੌਰ', 'ਸਿੰਘ', 'ਗਿਆ',
                     'ਦੀਆਂ', 'ਦਿੱਤਾ', 'ਜਿਸ', 'ਜਿਸ', 'ਕੀਤੀ', 'ਰਿਹਾ', 'ਕਿਹਾ', 'ਟੀ', 'ਰਹੀ', 'ਜਾ', 'ਹੋਏ', 'ਤਾਂ', 'ਹੋ', 'ਦੇ',
                     'ਨੂੰ', 'ਤੋਂ', 'ਪਰ', 'ਇਹ', 'ਹੀ', 'ਵ', 'ਦਾ', 'ਨੇ', 'ਕਿ', 'ਤੇ', 'ਦੀ', 'ਦੇ', 'ਨੂੰ', 'ਇਕ', 'ਬਾਰੇ',
                     'ਉਪਰੋਕਤ', 'ਭਰ', 'ਬਾਅਦ', 'ਬਾਅਦ', 'ਦੁਬਾਰਾ', 'ਵਿਰੁੱਧ', 'ਸਭ', 'ਲਗਭਗ', 'ਇਕੱਲੀ', 'ਨਾਲ', 'ਪਹਿਲਾਂ ਹੀ',
                     'ਵੀ', 'ਹਾਲਾਂ', 'ਹਮੇਸ਼ਾ', 'ਵਿਚ', 'ਵਿਚਕਾਰ', 'ਅਮਨਗਸਟ', 'ਮਾਤਰਾ', 'ਇਕ', 'ਅਤੇ', 'ਇਕ ਹੋਰ', 'ਕੋਈ',
                     'ਕਿਸੇ ਵੀ ਤਰ੍ਹਾਂ', 'ਕਿਸੇ ਵੀ', 'ਕੁਝ ਵੀ', 'ਕਿਸੇ ਵੀ ਜਗ੍ਹਾ', 'ਕਿਤੇ ਵੀ', 'ਹਨ', 'ਆਲੇ', 'ਬੈਕ', 'ਬਣ',
                     'ਬਣ ਗਿਆ', 'ਬਣਨ', 'ਬਣ', 'ਬਾਹਰੀ', 'ਇਲਾਵਾ', 'ਵਿਚਕਾਰ', 'ਪਰੇ', 'ਬਿੱਲ', 'ਦੋਨੋ', 'ਤਲ', 'ਪਰ', 'ਬਣ', 'ਕੇ',
                     'ਕਾਲ', 'ਕੈਨ', 'ਨਹੀਂ', 'ਕੈਨਟ', 'ਕੋ', 'ਕੰਪਿਊਟਰ', 'ਕੋਨ', 'ਸੀ', 'ਬਰਦਾਸ਼ਤ', 'ਰੋਏ', 'ਡੀ', 'ਵਰਣਨ',
                     'ਵਿਸਥਾਰ', 'ਕੀਤਾ', 'ਕਰਨਾ', 'ਕੀਤਾ', 'ਹੇਠਾਂ', 'ਹਰ', 'ਜਿਵੇਂ', 'ਅੱਠ', 'ਜਾਂ', 'ਹੋਰ', 'ਹੋਰ', 'ਖਾਲੀ',
                     'ਕਾਫ਼ੀ', 'ਆਦਿ', 'ਵੀ', 'ਕਦੇ', 'ਹਰ', 'ਹਰ ਕੋਈ', 'ਹਰ ਚੀਜ਼', 'ਹਰ ਜਗ੍ਹਾ', 'ਸਿਵਾਏ', 'ਕੁਝ', 'ਪੰਦਰਾਂ',
                     'ਪੰਜਾਹ', 'ਭਰਨ', 'ਲੱਭੋ', 'ਪਹਿਲਾ', 'ਪੰਜ', 'ਲਈ', 'ਪੁਰਾਣਾ', 'ਪਹਿਲਾਂ', 'ਚਾਲੀ', 'ਮਿਲਿਆ', 'ਚਾਰ', 'ਤੋ',
                     'ਫਰੰਟ', 'ਪੂਰਾ', 'ਅੱਗੇ', 'ਪ੍ਰਾਪਤ', 'ਦੇਣਾ', 'ਜਾਓ', 'ਹੈ', 'ਹੈ', 'ਹੈ ਨਾ', 'ਹੈ', 'ਉਹ', 'ਇਸ', 'ਉਸ',
                     'ਇੱਥੇ', 'ਅੱਗੇ', 'ਏਸ ਕੇ', 'ਏਥੇ', 'ਹੁਣ', 'ਉਸ', 'ਆਪ', 'ਉਸ', 'ਖ਼ੁਦ', 'ਉਸ', 'ਕਿਸ', 'ਪਰ', 'ਸੌ', 'ਮੈਂ',
                     'ਅਰਥਾਤ', 'ਜੇ', 'ਇਨ', 'ਅਸਲ', 'ਵਿਆਜ', 'ਇਨ', 'ਐੱਸ', 'ਇਹ', 'ਆਖ', 'ਬਾਅਦ', 'ਬਾਅਦ', 'ਘੱਟ', 'ਘੱਟ', 'ਲਿਡ',
                     'ਬਣਾਇਆ', 'ਬਹੁਤ ਸਾਰੇ', 'ਮੇ', 'ਮੈਂ', 'ਇਸ ਦੌਰਾਨ', 'ਸ਼ਾਇਦ', 'ਮਿੱਲ', 'ਮੇਰਾ', 'ਹੋਰ', 'ਹੋਰ', 'ਸਭ',
                     'ਜ਼ਿਆਦਾਤਰ', 'ਮੂਵ', 'ਬਹੁਤ', 'ਜ਼ਰੂਰ', 'ਮੇਰਾ', 'ਮੈਂ', 'ਨਾਮ', 'ਅਰਥਾਤ', 'ਨਾ', 'ਕਦੇ ਨਹੀਂ', 'ਫਿਰ', 'ਅਗਲਾ',
                     'ਨੌਂ', 'ਨਹੀਂ', 'ਕੋਈ ਨਹੀਂ', 'ਕੋਈ ਨਹੀਂ', 'ਨਾ', 'ਕੁਝ ਨਹੀਂ', 'ਹੁਣ', 'ਕਿਤੇ ਨਹੀਂ', 'ਦਾ', 'ਬੰਦ', 'ਅਕਸਰ',
                     'ਇਕ ਵਾਰ', 'ਇਕ', 'ਸਿਰਫ', 'ਉੱਤੇ', 'ਜਾਂ', 'ਹੋਰ', 'ਹੋਰ', 'ਹੋਰ', 'ਸਾਡਾ', 'ਸਾਡਾ', 'ਆਪ', 'ਬਾਹਰ', 'ਓਵਰ',
                     'ਖੁਦ', 'ਹਿੱਸਾ', 'ਉਹੀ', ' ਵੇਖੋ', 'ਜਾਪ ', 'ਵੇਖਣਾ', 'ਲੱਗਦਾ', 'ਗੰਭੀਰ', 'ਕਈ', 'ਉਹ', 'ਕਰਨਾ ਚਾਹੀਦਾ ਹੈ',
                     'ਸ਼ੋਅ', 'ਸਾਈਡ', 'ਏ', 'ਈਰਿਵਾਰ', 'ਛੇ', 'ਸੱਠ', 'ਉਸ', 'ਉਹ', 'ਉਨ੍ਹਾਂ', 'ਉਨ੍ਹਾਂ', 'ਆਪ', 'ਤਦ', 'ਤਦ ਤੋਂ',
                     'ਉੱਥੇ', 'ਉਸ ਤੋਂ ਬਾਅਦ', 'ਇਸ ਤਰ੍ਹਾਂ', 'ਇਸ ਲਈ', 'ਉਸ ਵਿਚ', 'ਇਸ ਤੋਂ ਬਾਅਦ', 'ਇਹ', 'ਉਹ', 'ਮੋਟੀ', 'ਪਤਲੇ',
                     'ਤੀਸਰਾ', 'ਇਹ', 'ਉਹ', 'ਭਾਵੇਂ', 'ਤਿੰਨ', 'ਤਿੰਨ', 'ਪੂਰੇ', 'ਥਰੂ', 'ਇਸ ਤਰ੍ਹਾਂ', 'ਇਕੋ', 'ਬਹੁਤ', 'ਚੋਟੀ',
                     'ਵੱਲ', 'ਵੱਲ', 'ਬਾਰਾਂ', 'ਵੀਹ', 'ਦੋ', 'ਅਣ', 'ਅਧੀਨ', 'ਤਦ', 'ਅਪ', 'ਉੱਤੇ', 'ਸਾਨੂੰ', 'ਬਹੁਤ', 'ਰਾਹ', 'ਸੀ',
                     'ਅਸੀਂ', 'ਚੰਗੀ', 'ਸਨ', 'ਕੀ', 'ਜੋ ਵੀ', 'ਕਦ', 'ਕਦੋਂ', 'ਜਦੋਂ ਵੀ', 'ਕਿੱਥੇ', 'ਬਾਅਦ', 'ਜਦੋਂ', 'ਜਿਸ ਵਿਚ ',
                     'ਜਿਸ ਵਿਚ', 'ਜਿਧਰ', 'ਕਿਤੇ ਵੀ', 'ਕੀ', 'ਕਿਹੜਾ', 'ਕਦ', 'ਕਿਥੇ', 'ਕੌਣ']


In [23]:
# hindi_stop=open("ranksnl-hindi.txt",'r')
# [i for i in hindi_stop]
hindi="""
   पर
इन
वह
यिह
वुह
जिन्हें
जिन्हों
तिन्हें
तिन्हों
किन्हों
किन्हें
इत्यादि
द्वारा
इन्हें
इन्हों
उन्हों
बिलकुल
निहायत
ऱ्वासा
इन्हीं
उन्हीं
उन्हें
इसमें
जितना
दुसरा
कितना
दबारा
साबुत
वग़ैरह
दूसरे
कौनसा
लेकिन
होता
करने
किया
लिये
अपने
नहीं
दिया
इसका
करना
वाले
सकते
इसके
सबसे
होने
करते
बहुत
वर्ग
करें
होती
अपनी
उनके
कहते
होते
करता
उनकी
इसकी
सकता
रखें
अपना
उसके
जिसे
तिसे
किसे
किसी
काफ़ी
पहले
नीचे
बाला
यहाँ
जैसा
जैसे
मानो
अंदर
भीतर
पूरा
सारा
होना
उनको
वहाँ
वहीं
जहाँ
जीधर
उनका
इनका
﻿के
हैं
गया
बनी
एवं
हुआ
साथ
बाद
लिए
कुछ
कहा
यदि
हुई
इसे
हुए
अभी
सभी
कुल
रहा
रहे
इसी
उसे
जिस
जिन
तिस
तिन
कौन
किस
कोई
ऐसे
तरह
किर
साभ
संग
यही
बही
उसी
फिर
मगर
का
एक
यह
से
को
इस
कि
जो
कर
मे
ने
तो
ही
या
हो
था
तक
आप
ये
थे
दो
वे
थी
जा
ना
उस
एस
पे
उन
सो
भी
और
घर
तब
जब
अत
व
न
"""

In [24]:
hindi.split('\n')

['',
 '   पर',
 'इन',
 'वह',
 'यिह',
 'वुह',
 'जिन्हें',
 'जिन्हों',
 'तिन्हें',
 'तिन्हों',
 'किन्हों',
 'किन्हें',
 'इत्यादि',
 'द्वारा',
 'इन्हें',
 'इन्हों',
 'उन्हों',
 'बिलकुल',
 'निहायत',
 'ऱ्वासा',
 'इन्हीं',
 'उन्हीं',
 'उन्हें',
 'इसमें',
 'जितना',
 'दुसरा',
 'कितना',
 'दबारा',
 'साबुत',
 'वग़ैरह',
 'दूसरे',
 'कौनसा',
 'लेकिन',
 'होता',
 'करने',
 'किया',
 'लिये',
 'अपने',
 'नहीं',
 'दिया',
 'इसका',
 'करना',
 'वाले',
 'सकते',
 'इसके',
 'सबसे',
 'होने',
 'करते',
 'बहुत',
 'वर्ग',
 'करें',
 'होती',
 'अपनी',
 'उनके',
 'कहते',
 'होते',
 'करता',
 'उनकी',
 'इसकी',
 'सकता',
 'रखें',
 'अपना',
 'उसके',
 'जिसे',
 'तिसे',
 'किसे',
 'किसी',
 'काफ़ी',
 'पहले',
 'नीचे',
 'बाला',
 'यहाँ',
 'जैसा',
 'जैसे',
 'मानो',
 'अंदर',
 'भीतर',
 'पूरा',
 'सारा',
 'होना',
 'उनको',
 'वहाँ',
 'वहीं',
 'जहाँ',
 'जीधर',
 'उनका',
 'इनका',
 '\ufeffके',
 'हैं',
 'गया',
 'बनी',
 'एवं',
 'हुआ',
 'साथ',
 'बाद',
 'लिए',
 'कुछ',
 'कहा',
 'यदि',
 'हुई',
 'इसे',
 'हुए',
 'अभी',
 'सभी',
 'कुल',
 'रहा',
 'रहे',
 'इसी',
 '

In [25]:
import csv
with open("hindi_stopwords.csv",'w') as file:
    writer=csv.writer(file)
    for word in hindi.split("\n"):
        writer.writerow([word])

In [36]:
d=pd.read_csv("punjabi_stopwords.csv",header=None)
d.columns=['words']
d.words

0           ਹੋਣ
1          ਆਪਣੇ
2          ਕਰਕੇ
3           ਲਿਆ
4          ਹੋਇਆ
5           ਤੱਕ
6            ਲੈ
7           ਰਹੇ
8         ਦੱਸਿਆ
9            ਪੀ
10          ਕੌਰ
11         ਸਿੰਘ
12          ਗਿਆ
13         ਦੀਆਂ
14        ਦਿੱਤਾ
15          ਜਿਸ
16          ਜਿਸ
17         ਕੀਤੀ
18         ਰਿਹਾ
19         ਕਿਹਾ
20           ਟੀ
21          ਰਹੀ
22           ਜਾ
23          ਹੋਏ
24          ਤਾਂ
25           ਹੋ
26           ਦੇ
27          ਨੂੰ
28          ਤੋਂ
29           ਪਰ
         ...   
271          ਦੋ
272          ਅਣ
273        ਅਧੀਨ
274          ਤਦ
275          ਅਪ
276        ਉੱਤੇ
277       ਸਾਨੂੰ
278        ਬਹੁਤ
279         ਰਾਹ
280          ਸੀ
281        ਅਸੀਂ
282        ਚੰਗੀ
283          ਸਨ
284          ਕੀ
285       ਜੋ ਵੀ
286          ਕਦ
287        ਕਦੋਂ
288     ਜਦੋਂ ਵੀ
289       ਕਿੱਥੇ
290        ਬਾਅਦ
291        ਜਦੋਂ
292    ਜਿਸ ਵਿਚ 
293     ਜਿਸ ਵਿਚ
294        ਜਿਧਰ
295     ਕਿਤੇ ਵੀ
296          ਕੀ
297       ਕਿਹੜਾ
298          ਕਦ
299        ਕਿਥੇ
300         ਕੌਣ
Name: words, Length: 301

In [14]:
np.sum(model.vectors)

-162298.98

In [9]:
search='ਇੱਕ'
search2='ਮੁੰਬਈ'
search3='ਡੱਡੂ'
search4='ਵਾਹਿਗੁਰੂ'

In [10]:
model.similar_by_word(search2)

[('ਮੁੰਬੲੀ', 0.7800889015197754),
 ('ਬੰਬਈ', 0.6970717310905457),
 ('ਚੇਨਈ', 0.631697416305542),
 ('ਇੰਦੌਰ', 0.6170452833175659),
 ('ਮੁਂਬਈ', 0.6128232479095459),
 ('ਮੁੰਗੇਰ', 0.5947357416152954),
 ('ਕੋਲਹਾਪੁਰ', 0.5882441997528076),
 ('ਮੁਮਬਈ', 0.5877441763877869),
 ('ਅਹਮਦਾਬਾਦ', 0.5833050012588501),
 ('ਫਰੀਦਾਬਾਦ', 0.5800901055335999)]

In [11]:
data=pd.read_csv("punjabi_newsss.csv")


In [12]:
data.head()

,Article_id,Article_title,Article_Content
0,1,NaN,ਮੁੰਬਈ ਬਿਊਰੋ ਬਾਲੀਵੁੱਡ ਅਕਸ਼ੈ ਕੁਮਾਰ ਨੇ ਮੁੰਬਈ ਹਾਲ ਹ...
1,2,ਹੁਣ ਮਾਈਕ੍ਰੋਸਾਫਟ ਵੀ ਬਦਲੇਗਾ ਪਿਸਤੌਲ ਇਮੋਜੀ,ਜਲੰਧਰ ਗੂਗਲ ਤੇ ਫੇਸਬੁੱਕ ਵੱਲੋਂ ਪਿਸਤੌਲ ਇਮੋਜੀ ਨੂੰ ਵ...
2,3,NaN,ਮੁੰਬਈ ਬਿਊਰੋ ਬਾਲੀਵੁੱਡ ਅਭਿਨੇਤਰੀ ਸੋਹਾ ਅਲੀ ਖਾਨ ਦੀ ...
3,4,ਟਰੈਕਟਰ ਤੇ ਮੋਟਰਸਾਈਕਲ ਦੀ ਟੱਕਰ ਜ਼ਖਮੀ,ਜੈਤੋ ਜਿੰਦਲ – ਬੀਤੇ ਦਿਨ ਰੋੜੀਕਪੂਰਾ ਤੋਂ ਕਾਸਮਭੱਟੀ ਰ...
4,5,ਸਾਲ ਕਰੋੜ ਰੁਪਏ ਪੂੰਜੀ ਵਪਾਰ ਕਰੇਗੀ ਮਾਰੂਤੀ ਸੁਜ਼ੂਕੀ,ਨਵੀਂ ਦਿੱਲੀ ਕਾਰ ਬਣਾਉਣ ਵਾਲੀ ਦੇਸ਼ ਦੀ ਤੋਂ ਵੱਡੀ ਕੰਪਨ...


In [13]:
# data.Article_Content
corpus=[]
for article in data.Article_Content:
    if type(article) == str:
#         if len(article)<10:
#             pass
        corpus.append(re.sub(" \d+",'',article))

In [14]:
tfid=TfidfVectorizer(stop_words=stopwords)

In [15]:
tfid_mat=tfid.fit_transform(corpus)

In [16]:
tfid_mat

<2969x5519 sparse matrix of type '<class 'numpy.float64'>'
	with 155015 stored elements in Compressed Sparse Row format>

In [17]:
len(corpus)

2969

In [18]:
tfid.vocabulary_

{'ਬਈ': 3910,
 'ਊਰ': 2102,
 'ਅਕਸ਼': 1201,
 'ਨਵ': 3674,
 'ਖਰ': 2583,
 'ਉਨ': 2002,
 'ਮਤ': 4154,
 'ਕਰ': 2435,
 'ਪਏ': 3723,
 'ਸਟ': 5075,
 'ਮਕ': 4114,
 'ਲਡ': 4702,
 'ਈਏ': 1830,
 'ਆਪਣ': 1582,
 'ਲਮ': 4752,
 'ਸਰ': 5199,
 'ਸਥ': 5117,
 'ਤਵ': 3377,
 'ਰਫ': 4506,
 'ਫਲ': 3892,
 'ਨਹ': 3703,
 'ਸਗ': 5064,
 'ਸਕਵ': 5058,
 'ਅਰ': 1348,
 'ਸਵ': 5258,
 'ਟਰ': 3102,
 'ਟਨ': 3088,
 'ਵਰਗ': 4870,
 'ਇਲ': 1782,
 'ਲਮਮ': 4760,
 'ਅਨ': 1276,
 'ਬਜ਼ਮ': 4053,
 'ਛਲ': 2889,
 'ਅਰਮ': 1375,
 'ਕਰਵ': 2469,
 'ਇਆ': 1688,
 'ਲਹ': 4796,
 'ਰਹ': 4600,
 'ਈਨ': 1881,
 'ਪਤਨ': 3760,
 'ਟਵ': 3134,
 'ਕਲ': 2478,
 'ਜਲ': 2997,
 'ਧਰ': 3551,
 'ਗਲ': 2712,
 'ਸਬ': 5161,
 'ਸਤ': 5104,
 'ਇਮ': 1768,
 'ਅਦ': 1266,
 'ਈਕ': 1834,
 'ਫਟ': 3862,
 'ਰਵ': 4564,
 'ਤਰ': 3336,
 'ਜਨ': 2940,
 'ਅਸ': 1416,
 'ਕਦਰ': 2398,
 'ਰਦਰਸ਼': 4473,
 'ਰਤ': 4449,
 'ਆਵ': 1653,
 'ਕਸ': 2511,
 'ਰਕ': 4327,
 'ਰਚਲ': 4378,
 'ਆਚ': 1536,
 'ਉਦ': 1996,
 'ਮਸ': 4261,
 'ਅਪਡ': 1299,
 'ਸ਼ਲ': 5016,
 'ਸਪ': 5144,
 'ਬਦਲ': 3965,
 'ਕਰਦ': 2448,
 'ਅਭ': 1322,
 'ਅਲ': 1385,
 'ਆਈ': 1473,
 'ਲਪ': 4730,
 'ਖੜ': 2605,
 'soha': 

In [19]:
tfid_mat.shape

(2969, 5519)

In [20]:
# fasttext_vocab_dict={}
# for word in tfid.vocabulary_:
#     if word in model.vocab:
#         fasttext_vocab_dict[word]=model.get_vector(word)


In [21]:
# len(fasttext_vocab_dict)

In [22]:
fasttext_vocab_matrix=[]
for word in tfid.vocabulary_:
    if word in model.vocab:
        fasttext_vocab_matrix.append(model.get_vector(word))
    else:
        fasttext_vocab_matrix.append(np.zeros((300)))

In [23]:
fasttext_vocab_matrix=np.array(fasttext_vocab_matrix)
#
# fasttext_vocab_matrix.reshape(fasttext_vocab_matrix.shape[0],1).shape

In [24]:
fasttext_vocab_matrix.shape

(5519, 300)

In [25]:
# print(tfid_mat)

In [26]:
tfid_mat.shape,fasttext_vocab_matrix.shape

((2969, 5519), (5519, 300))

In [27]:
vec=np.dot(tfid_mat.todense(),fasttext_vocab_matrix)

In [28]:
tfid_mat.todense().shape

(2969, 5519)

In [29]:
print(vec.shape)

(2969, 300)


In [30]:
re.sub(" \d+",'',corpus[0])

'ਮੁੰਬਈ ਬਿਊਰੋ ਬਾਲੀਵੁੱਡ ਅਕਸ਼ੈ ਕੁਮਾਰ ਨੇ ਮੁੰਬਈ ਹਾਲ ਹੀ ਨਵਾਂ ਖਰੀਦਿਆ ਹੈ। ਉਨ੍ਹਾਂ ਦੇ ਨਵੇਂ ਦੀ ਕੀਮਤ ਕਰੋੜ ਰੁਪਏ ਦੇ ਕਰੀਬ ਹੈ। ਉਨ੍ਹਾਂ ਨੇ ਅੰਧੇਰੀ ਵੈਸਟ ਦੇ ਟ੍ਰਾਂਸ ਕਾਨ ਟ੍ਰਾਯੰਫ ਨਾਮਕ ਬਿਲਡਿੰਗ ਖਰੀਦਿਆ ਹੈ। ਦੱਸ ਦੇਈਏ ਕਿ ਅਕਸ਼ੈ ਇੰਨੀ ਦਿਨੀਂ ਆਪਣੀ ਵਾਲੀ ਫਿਲਮ ਕੇਸਰੀ ਦੀ ਸ਼ੂਟਿੰਗ ਰੁੱਝੇ ਹੋਏ ਹਨ।ਵੀਂ ਮੰਜ਼ਿਲ ਤੇ ਅਕਸ਼ੈ ਦਾ ਨਵਾਂ ਅਕਸ਼ੈ ਦਾ ਅੰਧੇਰੀ ਸਥਿਤਵੀਂ ਮੰਜ਼ਿਲ ਤੇ ਹੈ। ਉਨ੍ਹਾਂ ਨੇ ਸਿਰਫ ਫਲੈਟ ਨਹੀਂ ਸਗੋਂ ਪੂਰਾ ਦਾ ਪੂਰਾ ਫਲੋਰ ਹੀ ਆਪਣੇ ਨਾਂ ਕੀਤਾ ਹੈ। ਅਕਸ਼ੈ ਨੇ ਹੀ ਫਲੋਰ ਦੇ ਫਲੈਟ ਖਰੀਦੇ ਹਨ। ਸਕਵੇਅਰ ਫਿੱਟ ਫੈਲੇ ਦੇ ਫਲੈਟ ਦੀ ਕੀਮਤ ਕਰੋੜ ਰੁਪਏ ਹੈ। ਸਵਿਮਿੰਗ ਪੂਲ ਜੌਗਿੰਗ ਟਰੈਕ ਫਿਟਨੈੱਸ ਸੈਂਟਰ ਵਰਗੀਆਂ ਸੁਵਿਧਾਵਾਂ ਅਕਸ਼ੈ ਦੇ ਹਨ। ਤੋਂ ਇਲਾਵਾ ਫਿਲਮਮੇਕਰ ਅਨੀਸ ਬਜ਼ਮੀ ਗਾਇਕ ਮੁਛਲ ਅਰਮਾਨ ਮਾਲਿਕ ਨੇ ਬਿਲਡਿੰਗ ਆਪਣਾ ਫਲੈਟ ਬੁੱਕ ਕਰਵਾਇਆ ਹੈ। ਅਕਸ਼ੈ ਫਿਲਹਾਲ ਜੁਹੂ ਰਹਿੰਦੇ ਹਨ। ਉਨ੍ਹਾਂ ਦੇ ਦਾ ਇੰਟੀਰਿਅਰ ਡਿਜ਼ਾਈਨ ਪਤਨੀ ਟਵਿੰਕਲ ਖੰਨਾ ਨੇ ਹੀ ਕੀਤਾ ਹੈ। ਉਨ੍ਹਾਂ ਨੇ ਜਿਸ ਬਿਲਡਿੰਗ ਖਰੀਦਿਆ ਮੰਜ਼ਿਲਾਂ ਦਾ ਹੈ।'

In [31]:
# big_array=[]
# for i in range(len(corpus)):
#     vec0=[]
#     for word in corpus[i].split():
#         if word in model.vocab:
#             vec0.append(model.get_vector(word))
#     big_array.append(np.sum(np.array(vec0),axis=0).reshape((300,1)))
    #     else:
#         vec0.append(np.zeros(300))

In [32]:
# big_array=np.array(big_array).reshape(2969,300)

In [33]:
# big_array.shape

In [34]:
# np.sum(np.array(vec0),axis=0).shape

In [35]:
# len(vec0),len(corpus[0].split())

In [31]:
len(model.vocab)

56850

In [32]:
vec.shape

(2969, 300)

In [33]:
cos_sims=cosine_similarity(vec)

In [34]:
cos_sims.shape

(2969, 2969)

In [35]:
cos_sims

array([[1.        , 0.64101639, 0.48861724, ..., 0.        , 0.68180757,
        0.76342966],
       [0.64101639, 1.        , 0.51010648, ..., 0.        , 0.62543864,
        0.45037971],
       [0.48861724, 0.51010648, 1.        , ..., 0.        , 0.46694684,
        0.44747601],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.68180757, 0.62543864, 0.46694684, ..., 0.        , 1.        ,
        0.53956674],
       [0.76342966, 0.45037971, 0.44747601, ..., 0.        , 0.53956674,
        1.        ]])

In [36]:
cos_sims[cos_sims>0.99]=0

In [37]:
# cos_sim_sorted[9][-5:]

In [38]:
cos_sim_sorted=cos_sims.argsort()

In [39]:
def top_5_similar(story_num,sims_sorted):
    indexes=sims_sorted[story_num][-5:]
#     print(indexes)
    similar_articles=[data.Article_Content[i] for i in indexes[::-1]]
    return similar_articles
  

In [40]:
corpus[196]

'24 ਨਵੀਂ ਦਿੱਲੀ ਪ੍ਰਧਾਨ ਮੰਤਰੀ ਨਰਿੰਦਰ ਮੋਦੀ ਦਿਨ ਦੇ ਚੀਨ ਦੌਰੇ ਤੋਂ ਸ਼ਨੀਵਾਰ ਸ਼ਾਮ ਨੂੰ ਵਾਪਿਸ ਦਿੱਲੀ ਪਹੁੰਚੇ। ਭਾਰਤ ਪਹੁੰਚਣ ਤੇ ਵਿਦੇਸ਼ ਮੰਤਰੀ ਸੁਸ਼ਮਾ ਸਵਰਾਜ ਨੇ ਪ੍ਰਧਾਨ ਮੰਤਰੀ ਮੋਦੀ ਦਾ ਸਵਾਗਤ ਕੀਤਾ। ਮੋਦੀ ਦੀ ਚੀਨ ਦੇ ਰਾਸ਼ਟਰਪਤੀ ਸ਼ੀ ਜਿਨਪਿੰਗ ਦੇ ਨਾਲ ਰਸਮੀ ਮੁਲਾਕਾਤ ਸੀ। ਦੋ ਦਿਨ ਦੋਵਾਂ ਨੇਤਾਵਾਂ ਨੇ ਕਰੀਬ ਘੰਟੇ ਇਕੱਠੇ ਗੁਜ਼ਾਰੇ। ਦੋਵਾਂ ਨੇਤਾਵਾਂ ਨੇ ਮੁੱਦਿਆਂ ਤੇ ਸਹਿਮਤੀ ਦੇ ਨਾਲ ਲੰਬੀ ਗੱਲਬਾਤ ਕੀਤੀ। ਪ੍ਰਧਾਨ ਮੰਤਰੀ ਮੋਦੀ ਦੇ ਦੌਰੇ ਦੇ ਦੂਸਰੇ ਦਿਨ ਦੀ ਸ਼ੁਰੂਆਤ ਪ੍ਰਸਿੱਧ ਲੇਕ ਦੇ ਕਿਨਾਰੇ ਸੈਰ ਦੇ ਨਾਲ ਹੋਈ। ਦੇ ਬਾਅਦ ਉਨ੍ਹਾਂ ਨੇ ਨੌਕਾ ਵਿਹਾਰ ਦਾ ਵੀ ਆਨੰਦ ਲਿਆ। ਭਾਰਤ ਦੇ ਪ੍ਰਧਾਨ ਮੰਤਰੀ ਨਰਿੰਦਰ ਮੋਦੀ ਅਤੇ ਚੀਨ ਦੇ ਰਾਸ਼ਟਰਪਤੀ ਸ਼ੀ ਜਿਨਪਿੰਗ ਸੀਮਾ ਵਿਵਾਦ ਨੂੰ ਲੈ ਕੇ ਡੋਕਲਾਮ ਜਿਹੇ ਦੇਸ਼ ਸੈਨਾ ਨੂੰ ਰਹੀਆਂ ਸਮੱਸਿਆਵਾਂ ਬਾਰੇ ਗੱਲਬਾਤ ਕੀਤੀ।'

In [41]:
top_5_similar(196,cos_sim_sorted)

['ਜਲੰਧਰ ਅਮਰੀਕੀ ਦਿੱਗਜ ਕੰਪਨੀ ਮਾਇਕ੍ਰੋਸਾਫਟ ਛੇ ਮਹੀਨਿਆਂ ਵਿੰਡੋਜ਼ 10 ਅਪਡੇਟ ਜਾਰੀ ਕਰਦੀ ਰਹਿੰਦੀ ਹੈ। ਉਥੇ ਹੀ ਜਾਣਕਾਰੀ ਦੇ ਮੁਤਾਬਕ ਮਾਇਕ੍ਰੋਸਾਫਟ ਨੇ ਵਿੰਡੋਜ਼ 10 ਅਪ੍ਰੈਲ ਜਾਰੀ ਹੋਣ ਵਾਲੇ ਨਵੀਂ ਅਪਡੇਟ ਦੀ ਘੋਸ਼ਣਾ ਦਿੱਤੀ ਹੈ ਜਿਸ ਨੂੰ 30 ਅਪ੍ਰੈਲ ਨੂੰ ਰੋਲਆਊਟ ਕੀਤਾ ਜਾਵੇਗਾ। ਕੰਪਨੀ ਦੁਆਰਾ ਪਹਿਲਾਂ ਅਪਡੇਟ ਨੂੰ 10 ਅਪ੍ਰੈਲ ਨੂੰ ਰਿਲੀਜ਼ ਦੇ ਯੋਜਨਾ ਬਣਾਈ ਜਾ ਰਹੀ ਸੀ ਆਧਿਕਾਰਤ ਲਾਂਚ ਤੋਂ ਪਹਿਲਾਂ ਕੰਪਨੀ ਅਪਡੇਟ ਕੁੱਝ ਹੋਰ ਸੁਧਾਰ ਕਰਣਾ ਚਾਹੁੰਦੀ ਹੈ। ਉਥੇ ਹੀ ਹੁਣ ਅਪਡੇਟ ਸੋਮਵਾਰ 30 ਅਪ੍ਰੈਲ ਨੂੰ ਮੈਨੂਅਲ ਡਾਊਨਲੋਡ ਉਪਲੱਬਧ ਹੋਵੇਗੀ। ਕਿ ਕੰਪਨੀ ਦੁਆਰਾ ਜਾਰੀ ਨਵੀਂ ਅਪਡੇਟ ਤੋਂ ਵਿੰਡੋਜ 10 ਦੇ ਯੂਜ਼ਰਸ ਨੂੰ ਕੁਝ ਨਵੇਂ ਫੀਚਰ ਮਿਲਣਗੇ। ਜਿਸ ਨਵੀਂ ਟਾਈਮਲਾਈਨ ਸਮਾਰਟ ਹੋਮ ਕੰਟਰੋਲ ਆਦਿ ਸ਼ਾਮਿਲ ਹੋਣਗੇ।',
 '20 ਨਵੀਂ ਦਿੱਲੀ ਗਲੋਬਲ ਬਾਜ਼ਾਰਾਂ ਤੋਂ ਮਿਲੇ ਮਜ਼ਬੂਤੀ ਦੇ ਸੰਕੇਤਾਂ ਵਿਚਕਾਰ ਅੱਜ ਭਾਰਤੀ ਸ਼ੇਅਰ ਬਾਜ਼ਾਰ ਵੱਡੀ ਤੇਜ਼ੀ ਵਾਧੇ ਨਾਲ ਬੰਦ ਹੋਇਆ। ਕਾਰੋਬਾਰ ਦੇ ਅੰਤ ਅੱਜ ਸੈਂਸੈਕਸ 190 66 ਅੰਕ ਯਾਨੀ 0 55 ਫੀਸਦੀ ਕੇ 35 160 36 ਤੇ ਅਤੇ ਨਿਫਟੀ 47 05 ਅੰਕ ਯਾਨੀ 0 44 ਫੀਸਦੀ ਕੇ 10 739 35 ਤੇ ਬੰਦ ਹੋਇਆ। ਕਾਰੋਬਾਰ ਦੇ ਦੌਰਾਨ ਅੱਜ ਸੈਂਸੈਕਸ 35213 ਅਤੇ ਨਿਫਟੀ 10759 ਦੇ ਪੱਧਰ ਤੇ ਪਹੁੰਚਣ ਕਾਮਯਾਬ ਹੋ ਗਿਆ। 24 ਮਿਡ ਸਮਾਲਕੈਪ ਸ਼ੇਅਰਾਂ ਵਿਚ ਵਾਧਾ 20 ਮਿਡਕੈਪ ਅਤੇ ਸਮਾਲਕੈਪ ਸ਼ੇਅਰਾਂ ਵਿਚ ਵੀ ਵਾਧਾ ਦੇਖਣ ਨੂੰ ਮਿਲਿਆ। ਬੀ ਐੱਸ ਦੇ 

In [42]:
##################  projection part ####################

In [44]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector

/home/gurmukh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [43]:
# big_array.shape
big_array=[]
for i in range(len(corpus)):
    vec0=[]
    for word in corpus[i].split():
        if word in model.vocab:
            vec0.append(model.get_vector(word))
    big_array.append(np.sum(np.array(vec0),axis=0).reshape((300,1)))
    #     else:
#         vec0.append(np.zeros(300))
big_array=np.array(big_array).reshape(2969,300)

In [92]:
# embedding_variable=tf.Variable(big_array,name="embedding")

In [94]:
embedding_variable=tf.Variable(vec,name="embedding")

In [52]:
file=open("metadata.tsv",'w')
for news in corpus:
    file.write(news + "\n")
file.close()

In [53]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())

In [95]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
# create a TensorFlow summary writer
    summary_writer = tf.summary.FileWriter('embed_visual', sess.graph)
    config = projector.ProjectorConfig()
    embedding_conf = config.embeddings.add()
    embedding_conf.tensor_name = 'embedding'
    embedding_conf.metadata_path = 'metadata.tsv'
    projector.visualize_embeddings(summary_writer, config)

    # save the model
    saver = tf.train.Saver()
    saver.save(sess, 'embed_visual/model.ckpt')

In [45]:
###### clustering the embeddings ##########

In [46]:
from sklearn.cluster import KMeans

In [92]:
k=KMeans(n_clusters=5)

In [93]:
result=k.fit_transform(vec)

In [69]:
k.labels_ ==1

array([ True, False,  True, ..., False,  True, False])

In [95]:
np.argwhere([k.labels_ ==2])[:,1]

array([  10,   16,   24,   27,   32,   43,   51,   56,   64,   69,   87,
         93,  100,  106,  115,  116,  121,  135,  136,  146,  147,  153,
        158,  182,  197,  200,  201,  220,  222,  227,  241,  251,  256,
        258,  259,  266,  268,  273,  280,  294,  298,  301,  322,  327,
        332,  335,  340,  352,  355,  363,  378,  388,  401,  405,  410,
        411,  420,  422,  432,  433,  436,  440,  442,  443,  444,  445,
        447,  454,  461,  474,  479,  485,  488,  489,  494,  499,  514,
        520,  523,  525,  533,  543,  544,  551,  553,  560,  562,  563,
        570,  576,  590,  595,  596,  597,  603,  608,  619,  621,  623,
        635,  639,  649,  653,  655,  668,  669,  681,  684,  687,  705,
        706,  709,  710,  711,  713,  716,  718,  737,  739,  743,  745,
        746,  754,  759,  763,  773,  775,  783,  795,  810,  824,  838,
        840,  841,  875,  888,  905,  916,  919,  920,  923,  930,  939,
        941,  943,  954,  955,  956,  961,  972,  9

In [96]:
[data.Article_Content[i] for i in np.argwhere([k.labels_ ==2])[:,1]]

['ਨਵੀਂ ਦਿੱਲੀ ਸਾਲ ਕੌਮਾਂਤਰੀ ਉਡਾਣਾਂ ਸ਼ੁਰੂ ਦਾ ਪਲਾਨ ਬਣਾ ਰਹੀ ਵਿਸਤਾਰਾ ਅਹਿਮ ਗਲੋਬਲ ਏਅਰਲਾਈਨ ਗਰੁੱਪ ਦਾ ਹਿੱਸਾ ਹੈ। ਟਾਟਾ ਸੰਨਜ਼ ਅਤੇ ਸਿੰਗਾਪੁਰ ਏਅਰਲਾਈਨ ਦੀ ਅਗਵਾਈ ਵਾਲੀ ਕੰਪਨੀ ਨੇ ਕੌਮਾਂਤਰੀ ਹਵਾਬਾਜ਼ੀ ਟਰਾਂਸਪੋਰਟ ਸੰਗਠਨ ਟੀ ਨਾਲ ਜੁੜਨ ਦੀ ਜਾਣਕਾਰੀ ਦਿੱਤੀ ਹੈ। ਹੁਣ ਸੰਗਠਨ ਸਿਰਫ ਦੋ ਭਾਰਤੀ ਜਹਾਜ਼ ਕੰਪਨੀਆਂ ਇੰਡੀਆ ਅਤੇ ਜੈੱਟ ਏਅਰਵੇਜ਼ ਹੀ ਸ਼ਾਮਲ ਸਨ। ਟੀ ਦੇ ਕੁੱਲ 280 ਮੈਂਬਰ ਜਿਸ ਹੁਣ ਵਿਸਤਾਰਾ ਵੀ ਸ਼ਾਮਲ ਹੋ ਹੈ। ਹਵਾਬਾਜ਼ੀ ਕੰਪਨੀਆਂ ਦੇ ਗਲੋਬਲ ਗਰੁੱਪ ਸ਼ਾਮਲ ਹੋਣ ਨਾਲ ਵਿਸਤਾਰਾ ਨੂੰ ਕੌਮਾਂਤਰੀ ਕੰਪਨੀਆਂ ਨਾਲ ਕੋਡ ਸਾਂਝਾ ਅਤੇ ਇੰਟਰਲਾਈਨ ਸਮਝੌਤੇ ਮਿਲੇਗੀ। ਕੰਪਨੀ ਨੇ ਕਿਹਾ ਕਿ ਨਾਲ ਭਾਰਤ ਜਾਣ ਵਾਲੇ ਮੁਸਾਫਰਾਂ ਨੂੰ ਬਿਹਤਰ ਸਰਵਿਸ ਮੁਹੱਈਆ ਕਰਾ ਸਕੇਗੀ। ਵਿਸਤਾਰਾ ਕੰਪਨੀ ਨੇ ਗਰੁੱਪ ਸ਼ਾਮਲ ਹੋਣ ਤੇ ਕਿਹਾ ਕਿ ਸਾਨੂੰ ਕੌਮਾਂਤਰੀ ਭਾਈਚਾਰੇ ਵਾਲੇ ਟੀ ਗਰੁੱਪ ਸ਼ਾਮਲ ਹੋਣ ਤੇ ਮਾਣ ਮਹਿਸੂਸ ਹੋ ਰਿਹਾ ਹੈ। ਕੰਪਨੀ ਦੇ ਸੀ ਨੇ ਕਿਹਾ ਕਿ ਟੀ ਨਾਲ ਜੁੜਨ ਦਾ ਤੋਂ ਚੰਗਾ ਸਮਾਂ ਕੀ ਹੋ ਸਕਦਾ ਹੈ ਕਿਉਂਕਿ ਅਸੀਂ ਹੀ ਕੌਮਾਂਤਰੀ ਉਡਾਣਾਂ ਸ਼ੁਰੂ ਦੇ ਨੇੜੇ ਪਹੁੰਚ ਹਾਂ। ਜ਼ਿਕਰਯੋਗ ਹੈ ਕਿ ਮੌਜੂਦਾ ਸਮੇਂ ਵਿਸਤਾਰਾ ਹਫਤੇ 730 ਫਲਾਈਟਾਂ ਦਾ ਸੰਚਾਲਨ ਕਰਦੀ ਹੈ। ਦੇ ਬੇੜੇ 20 320 ਜਹਾਜ਼ ਹਨ।',
 'ਸਿਡਨੀ ਸਿਡਨੀ ਦੇ ਉਤਰੀ ਪੱਛਮੀ ਇਲਾਕੇ ਵਿਚ ਦੋ ਗੱਡੀਆਂ ਦੀ ਆਪਸੀ ਟੱਕਰ ਤੋਂ ਬਾਅਦ 19 ਸਾਲਾ ਕੁੜੀ ਦੀ ਮੌਤ ਹੋ ਅਤੇ ਦੇ 58 ਸਾਲਾ ਪਿਤਾ ਗੰਭੀਰ ਰੂਪ ਨਾਲ ਜ਼ਖਮੀ 

In [77]:
############# lda model ###################

In [20]:
import gensim
from gensim import corpora

In [23]:
dictionary=corpora.Dictionary([i.split() for i in corpus])

In [25]:
print(dictionary)

Dictionary(32501 unique tokens: ['ਅਕਸ਼ੈ', 'ਅਨੀਸ', 'ਅਰਮਾਨ', 'ਅੰਧੇਰੀ', 'ਆਪਣਾ']...)


In [28]:
doc_term_matrix = [dictionary.doc2bow(doc.split()) for doc in corpus]

In [31]:
# doc_term_matrix

In [32]:
Lda=gensim.models.ldamodel.LdaModel

In [36]:
ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=5)

In [35]:
print(ldamodel.print_topics(num_topics=10, num_words=3))


[(0, '0.020*"ਮੰਡੀ" + 0.016*"ਖਰੀਦ" + 0.015*"ਲਿਫਟਿੰਗ"'), (1, '0.034*"ਦੇ" + 0.025*"ਨੂੰ" + 0.020*"ਦੀ"'), (2, '0.026*"ਹੈ।" + 0.018*"ਨੇ" + 0.018*"ਦੀ"'), (3, '0.031*"ਦੀ" + 0.029*"ਹੈ।" + 0.022*"ਦੇ"'), (4, '0.029*"ਹੈ।" + 0.026*"ਦੇ" + 0.024*"ਦੀ"'), (5, '0.023*"ਦੇ" + 0.011*"ਨੇ" + 0.010*"ਸਥਾਨ"'), (6, '0.044*"ਸਿੰਘ" + 0.023*"ਨੇ" + 0.023*"ਦੇ"'), (7, '0.017*"ਤੇ" + 0.016*"ਤੋਂ" + 0.015*"ਨੂੰ"'), (8, '0.030*"ਨੂੰ" + 0.030*"ਦੇ" + 0.025*"ਦੀ"'), (9, '0.037*"ਦੇ" + 0.028*"ਦੀ" + 0.027*"ਨੂੰ"')]


In [43]:
43.3%print(ldamodel.show_topics())

[(0, '0.026*"ਦੇ" + 0.021*"ਨੂੰ" + 0.018*"ਦੀ" + 0.016*"ਸਿੰਘ" + 0.016*"ਅਤੇ" + 0.013*"ਤੇ" + 0.013*"ਹੈ।" + 0.012*"ਦਾ" + 0.011*"ਕਿ" + 0.011*"ਨੇ"'), (1, '0.030*"ਦੇ" + 0.023*"ਨੂੰ" + 0.022*"ਦੀ" + 0.022*"ਨੇ" + 0.019*"ਹੈ।" + 0.016*"ਤੇ" + 0.015*"ਦਾ" + 0.014*"ਹੈ" + 0.012*"ਕਿ" + 0.012*"ਅਤੇ"'), (2, '0.023*"ਦੇ" + 0.022*"ਨੂੰ" + 0.021*"ਦੀ" + 0.017*"ਵਿਚ" + 0.017*"ਹੈ" + 0.017*"ਦਾ" + 0.015*"ਹੈ।" + 0.013*"ਅਤੇ" + 0.013*"ਵੀ" + 0.013*"ਤੇ"'), (3, '0.027*"ਦੇ" + 0.024*"ਨੇ" + 0.023*"ਦੀ" + 0.023*"ਸਿੰਘ" + 0.023*"ਨੂੰ" + 0.019*"ਤੇ" + 0.015*"ਕੇ" + 0.014*"ਪੁਲਸ" + 0.014*"ਕਿ" + 0.011*"ਹੈ।"'), (4, '0.038*"ਦੇ" + 0.028*"ਦੀ" + 0.027*"ਨੂੰ" + 0.024*"ਨੇ" + 0.017*"ਹੈ।" + 0.017*"ਕਿ" + 0.017*"ਤੇ" + 0.015*"ਦਾ" + 0.013*"ਅਤੇ" + 0.012*"ਤੋਂ"')]
